In [1]:
%load_ext autoreload
%autoreload 2

In [129]:
import numpy as np

import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.colors import ListedColormap
import webcolors
from PIL import Image, ImageDraw
from scipy.cluster.hierarchy import DisjointSet

import networkx as nx

from minimal.imaging import blit_rooms, draw_sep_nask_wireframe
from minimal.layout import NodeType, NODE_COLOR
from minimal.gen import PlanMasks
from minimal.rooms import RoomAreas, extract_rooms
from minimal.rooms import create_room_mask, create_inv_room_mask, scale_room_mask
from minimal.walls import create_sep_mask, scale_sep_mask, _conv_mask
from minimal.doors import extract_face_walls, candidate_wall_runs
from minimal.doors import select_rooms_to_join

%matplotlib inline

In [25]:
%%time
pm = PlanMasks.create_from_state(torch.load("data/plan_masks_02.pth"))
rooms = extract_rooms(pm)
rooms.sort(key=lambda r: r.total_area())
# blit_rooms(rooms)

CPU times: user 939 ms, sys: 0 ns, total: 939 ms
Wall time: 938 ms


In [139]:
%%time

# 1-based index
rooms_to_join = select_rooms_to_join(rooms, pm.graph)
rooms_to_join

CPU times: user 414 µs, sys: 34 µs, total: 448 µs
Wall time: 458 µs


[(7, 4), (8, 4), (9, 3), (8, 1), (9, 5), (8, 6), (9, 7), (5, 2)]

In [134]:
room_mask = create_room_mask(rooms)
sep_mask = create_sep_mask(room_mask)

sx = 3
sy = 3

sep_mask = scale_sep_mask(sep_mask, sx, sy)
room_mask = scale_room_mask(room_mask, sx, sy)

inv_room_mask = create_inv_room_mask(room_mask, len(rooms))

In [136]:
face_walls = extract_face_walls(sep_mask)

In [173]:
for ra, rb in rooms_to_join:
    cruns = candidate_wall_runs(face_walls, room_mask, inv_room_mask, ra, rb)
    print(ra, rb, cruns)

7 4 [(146, 115, 49, 'h')]
8 4 [(148, 113, 19, 'v')]
9 3 [(69, 91, 28, 'h')]
8 1 [(167, 73, 40, 'h')]
9 5 [(69, 43, 46, 'h')]
8 6 [(117, 70, 43, 'h')]
9 7 [(104, 118, 46, 'h')]
5 2 [(27, 43, 46, 'h')]
